In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=720)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# 全撈 
one_year_ago_str = '20200101'

# SQL 查詢 加權指數
query_twse = f"""
SELECT *
FROM daily_twse
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# SQL 查詢 個股
query_insti_inv = f"""
SELECT *
FROM daily_insti_inv
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
twse_df_raw = pd.read_sql(query_twse, engine)

stock_df_raw = pd.read_sql(query_stock, engine)

insti_inv_raw = pd.read_sql(query_insti_inv, engine)

sub_category_list = pd.read_sql(query_sub_category, engine)

stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])

insti_inv_raw['Date'] = pd.to_datetime(insti_inv_raw['Date'])

In [ ]:
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
stock_df_year = stock_df_raw[(stock_df_raw['Date']>'2024-01-01' ) & (stock_df_raw['Date']<'2024-12-07' )]
# filtered_df = stock_df_year[stock_df_year['證券代號'] == '3715']
# filtered_df['20RS_rank'].head(50)
filtered_df = stock_df_year[stock_df_year['證券代號'] == '6584']
filtered_df.loc[:, '60RS_rank_mean'] = filtered_df['60RS_rank'].rolling(30).mean()
print(filtered_df)

# 60日線的RS千萬不能低於80


import matplotlib.pyplot as plt

# 過濾出符合條件的資料
filtered_df = stock_df_year[stock_df_year['證券代號'] == '6584']
# 如果日期欄位不是索引，將其設定為索引
filtered_df = filtered_df.set_index('Date')
# 繪製折線圖
plt.figure(figsize=(10, 6))
plt.plot(filtered_df.index, filtered_df['20RS_rank'], label='20RS_rank')
plt.plot(filtered_df.index, filtered_df['60RS_rank'], label='60RS_rank')
plt.plot(filtered_df.index, filtered_df['240RS_rank'], label='240RS_rank')

# 標題和標籤
plt.title('RS Rank Trend for Stock 2439')
plt.xlabel('Date')
plt.ylabel('Rank Value')
plt.legend()

# 顯示圖表
plt.grid(True)
plt.show()


In [7]:
# # 驗證一下60RS 都大於80的股票是不是表現很好

# today_str = '2023-01-03'
# stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
# stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

# today_list = stock_df_today[(stock_df_today['成交金額']>500000000)]

# # 計算一年內高點
# stock_id_list = today_list['證券代號'].unique().tolist()

# stock_id_filter = []
# for i in stock_id_list:
#         if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
#                 #print(i)
#                 # 計算個股 RS
#                 temp_df = stock_df_raw[stock_df_raw['證券代號']==i].copy()
#                 temp_df.loc[:, '60RS_rank_min'] = temp_df['60RS_rank'].rolling(30).min()
#                 RS_rank_min = temp_df[temp_df['Date'] == today_str]['60RS_rank_min'].iloc[0]

#                 if(RS_rank_min > 80):
#                     stock_id_filter.append(i) 


# filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
# filter_stock_today

In [67]:
# 這樣是可以撈到華城 可以回測多筆這樣績效卻沒有很好


# day_list = stock_df_raw[(stock_df_raw['Date']>'2022-12-04' ) & (stock_df_raw['Date']<'2022-12-07' )]
# day_list = day_list['Date'].unique().tolist()
# filter_stock_list = pd.DataFrame(columns=['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價'])


# for day in day_list:
#         today_str = day.strftime('%Y-%m-%d')
#         stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
#         stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

#         # 收盤在季線以上 年線以上 成值 1億 
#         today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['60MA']) & 
#                                 (stock_df_today['60MA']>stock_df_today['240MA']) & 
#                                 (stock_df_today['成交金額']>200000000) &
#                                 (stock_df_today['成交股數']>2000000) & 
#                                 (stock_df_today['60RS_rank']>75) ]

#         # 計算一年內高點
#         stock_id_list = today_list['證券代號'].unique().tolist()
        
#         stock_id_filter = []
#         for i in stock_id_list:
#                 if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天

#                         # 計算個股 RS
#                         today_df = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] == today_str) ].copy()
#                         close = today_df['收盤價'].iloc[0]

#                         #print(i)
#                         # 計算個股 RS
#                         temp_df_his = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] < today_str)  ].copy()
#                         # 一年內高低點
#                         temp_df_his.loc[:, '240DAY_min'] = temp_df_his['收盤價'].rolling(240).min()
#                         temp_df_his.loc[:, '240DAY_max'] = temp_df_his['收盤價'].rolling(240).max()
#                         temp_df_his.loc[:, '240DAY_money'] = temp_df_his['成交金額'].rolling(120).mean()

#                         DAY240_min = temp_df_his.tail(1)['240DAY_min'].iloc[0]
#                         DAY240_max = temp_df_his.tail(1)['240DAY_max'].iloc[0]
#                         DAY240_money = temp_df_his.tail(1)['240DAY_money'].iloc[0]
#                         MA60 = temp_df_his.tail(1)['60MA'].iloc[0] 
                        
#                         # 計算個股 60日RS rank 從原本小於 80 飆上來 
#                         temp_df_his.loc[:, '60RS_rank_mean'] = temp_df_his['60RS_rank'].rolling(30).mean()
#                         RS_rank_mean = temp_df_his.tail(1)['60RS_rank_mean'].iloc[0]


#                         # RS_rank_min = 90
#                         #距離一年高點25%以內
#                         #距離一年低點20%以上
#                         if( (close*1.2 >  DAY240_max) & (close >  DAY240_min*1.2) & (RS_rank_mean <  50) & (DAY240_money < 150000000) & (close < MA60*1.3)):
#                                 stock_id_filter.append(i) 


#         filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
#         filter_stock_today = filter_stock_today[['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價']]
#         if (len(filter_stock_today)!=0):
#                 filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)



# filter_stock_list

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價
0,2022-12-06,1519,華城,16004923,6358,770650186,47.2,48.7


In [208]:
# import pandas as pd
# from multiprocessing import Pool

# def process_day(day, stock_df_raw):
#     today_str = day.strftime('%Y-%m-%d')
#     stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
#     stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

#     # 收盤在季線以上 年線以上 成值 2億
#     today_list = stock_df_today[(stock_df_today['收盤價'] > stock_df_today['60MA']) & 
#                                 (stock_df_today['60MA'] > stock_df_today['240MA']) & 
#                                 (stock_df_today['成交金額'] > 200000000) &
#                                 (stock_df_today['成交股數'] > 2000000) & 
#                                 (stock_df_today['60RS_rank'] > 75)]

#     stock_id_list = today_list['證券代號'].unique().tolist()
#     stock_id_filter = []

#     for i in stock_id_list:
#         if len(i) == 4:
#             today_df = stock_df_raw[(stock_df_raw['證券代號'] == i) & (stock_df_raw['Date'] == today_str)].copy()
#             close = today_df['收盤價'].iloc[0]

#             temp_df_his = stock_df_raw[(stock_df_raw['證券代號'] == i) & (stock_df_raw['Date'] < today_str)].copy()

#             # 計算一年內高低點
#             DAY240_min = temp_df_his['收盤價'].rolling(240).min().iloc[-1]
#             DAY240_max = temp_df_his['收盤價'].rolling(240).max().iloc[-1]
#             DAY240_money = temp_df_his['成交金額'].rolling(120).mean().iloc[-1]
#             MA60 = temp_df_his['60MA'].iloc[-1]

#             RS_rank_mean = temp_df_his['60RS_rank'].rolling(30).mean().iloc[-1]

#             # RS_rank_min = 90
#             #距離一年高點25%以內 距離一年低點20%以上
#             if (close * 1.2 > DAY240_max) and (close > DAY240_min * 1.2) and (RS_rank_mean < 50) and (DAY240_money < 150000000) and (close < MA60 * 1.3):
#                 stock_id_filter.append(i)

#     filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
#     filter_stock_today = filter_stock_today[['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價']]
    
#     return filter_stock_today if not filter_stock_today.empty else None


In [ ]:
# if __name__ == '__main__':
#     day_list = stock_df_raw[(stock_df_raw['Date']>'2022-12-04' ) & (stock_df_raw['Date']<'2022-12-07' )]
#     day_list = day_list['Date'].unique().tolist()

#     with Pool(processes=4) as pool:  # 調整 processes 來匹配 CPU 核心數
#         results = pool.starmap(process_day, [(day, stock_df_raw) for day in day_list])

#     # 將所有結果合併
#     filter_stock_list = pd.concat([res for res in results if res is not None], ignore_index=True)


個股條件測試區

In [147]:
stock_df_today = stock_df_raw[(stock_df_raw['Date']>='2023-03-11' ) & (stock_df_raw['Date']<'2023-03-30' ) & (stock_df_raw['證券代號'] == '3231') ]

tt = stock_df_today[(stock_df_today['收盤價']>stock_df_today['5MA']) & 
                    (stock_df_today['5MA']>stock_df_today['10MA']) & 
                    (stock_df_today['10MA']>stock_df_today['60MA']) & 
                    (stock_df_today['60MA']>stock_df_today['240MA']) & 
                    (stock_df_today['成交金額']>1000000000) &
                    (stock_df_today['成交筆數']>10000) & 
                    (stock_df_today['60RS_rank']>75) &
                    (stock_df_today['20RS_rank']>85) ]


temp_df_his = stock_df_raw[(stock_df_raw['證券代號']=='3231') & (stock_df_raw['Date'] <'2023-03-15') & (stock_df_raw['收盤價'] != 0) ].copy()
temp_df_his.loc[:, '60MAV'] = temp_df_his['成交筆數'].rolling(60).mean()

# 季內高低點
temp_df_his.loc[:, 'backtest_dayDAY_min'] = temp_df_his['收盤價'].rolling(60).min()
temp_df_his.loc[:, 'backtest_dayDAY_max'] = temp_df_his['收盤價'].rolling(60).max()
DAYbacktest_day_min = temp_df_his.tail(1)['backtest_dayDAY_min'].iloc[0]
DAYbacktest_day_max = temp_df_his.tail(1)['backtest_dayDAY_max'].iloc[0]

print(temp_df_his['收盤價'].rolling(60).min().tail(1))
print(temp_df_his['收盤價'].rolling(60).max().tail(1))

print(DAYbacktest_day_min)


print(abs(DAYbacktest_day_max - DAYbacktest_day_min) / DAYbacktest_day_min )

backtest_60MAV = temp_df_his.tail(1)['60MAV'].iloc[0]
print(backtest_60MAV)

1358670    27.3
Name: 收盤價, dtype: float64
1358670    33.95
Name: 收盤價, dtype: float64
27.3
0.24358974358974367
6164.466666666666


In [141]:
tt

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,60MA,60RS,20RS_sd,20RS_rank,60RS_sd,60RS_rank,240MA,240RS,240RS_sd,240RS_rank
1360461,2023-03-15,3231,緯創,32787602,16013,1130588861,34.00,34.85,33.95,34.50,...,30.9700,111.3981,1.068553,88.609715,1.088994,81.183696,28.3463,121.7092,1.195187,84.254606
1365835,2023-03-20,3231,緯創,28645100,13347,1019862791,35.00,35.90,35.00,35.90,...,31.3350,114.5684,1.090384,92.578125,1.120366,85.881696,28.4125,126.3528,1.235827,87.779018
1367627,2023-03-21,3231,緯創,67075620,29866,2500129104,37.00,37.95,36.55,37.55,...,31.4858,119.2600,1.123767,95.147797,1.160372,90.239822,28.4446,132.0111,1.282623,89.737869
1369420,2023-03-22,3231,緯創,50786897,23001,1930533384,38.45,38.45,37.30,38.20,...,31.6408,120.7301,1.115241,95.092025,1.157845,90.184049,28.4790,134.1341,1.282200,90.686001
1371213,2023-03-23,3231,緯創,79210133,36220,3102975482,38.20,39.80,37.85,39.65,...,31.8158,124.6235,1.137849,96.653653,1.188859,93.028444,28.5200,139.0252,1.319669,92.303402
1373006,2023-03-24,3231,緯創,69939632,29215,2766859558,40.20,40.35,38.65,40.00,...,31.9942,125.0228,1.132404,96.488294,1.190411,93.422520,28.5623,140.0448,1.324440,92.140468
1374800,2023-03-27,3231,緯創,68690410,28601,2809346312,40.00,41.35,39.85,41.20,...,32.1808,128.0265,1.158353,96.935933,1.227264,94.484680,28.6100,144.0056,1.368452,93.314763


In [11]:
import warnings
# 計算出場  使用60RS看看
# 忽略 FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
day_list = stock_df_raw[(stock_df_raw['Date']>='2023-10-10' ) & (stock_df_raw['Date'] <= '2024-12-17' )]
day_list = day_list['Date'].unique().tolist()

backtest_day = 60

# 初始化 filter_stock_list 和 stock_id_filter
filter_stock_list = pd.DataFrame(columns=['Date', '證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '收盤價', '明天開盤價', '明天收盤價', '明今差'])
stock_id_filter = set()  # 使用集合來存儲 stock_id，查詢效率更高

# 遍歷日期列表
for day in day_list:

    today_str = day.strftime('%Y-%m-%d')
    
    stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

    # 收盤在季線以上 年線以上 成值 10億 
    today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['5MA']) & 
                                (stock_df_today['5MA']>stock_df_today['10MA']) & 
                                (stock_df_today['10MA']>stock_df_today['60MA']) & 
                                (stock_df_today['60MA']>stock_df_today['240MA']) & 
                                (stock_df_today['成交金額']>1000000000) &
                                (stock_df_today['成交筆數']>10000) & 
                                (stock_df_today['60RS_rank']>75) &
                                (stock_df_today['20RS_rank']>85) ]

    # 計算一年內高點
    stock_id_list = today_list['證券代號'].unique().tolist()

    for i in stock_id_list:
        if len(i) == 4 and i not in stock_id_filter:  # 確保是新 stock_id
            today_df = stock_df_today[stock_df_today['證券代號'] == i]
            close = today_df['收盤價'].iloc[0]
            open = today_df['開盤價'].iloc[0]

            if( close < open):
                continue

            volumn = today_df['成交筆數'].iloc[0]

            temp_df_his = stock_df_raw[
                (stock_df_raw['證券代號'] == i) & 
                (stock_df_raw['Date'] < today_str) & 
                (stock_df_raw['收盤價'] != 0)
            ].copy()

            temp_df_fut = stock_df_raw[
                (stock_df_raw['證券代號'] == i) & 
                (stock_df_raw['Date'] > today_str) & 
                (stock_df_raw['收盤價'] != 0)
            ].copy()

            if not temp_df_fut.empty:
                tom_close = temp_df_fut['收盤價'].iloc[0]
                tom_open = temp_df_fut['開盤價'].iloc[0]
                tom_diff = round(100 * (tom_close - close) / close , 2) 
            else :
                tom_close = 0
                tom_open = 0
                tom_diff = 0

            if temp_df_his.empty:  # 避免 temp_df_his 為空時報錯
                continue

            temp_df_his['60MAV'] = temp_df_his['成交筆數'].rolling(60).mean()

            # 計算高低點和均值
            DAYbacktest_day_min = temp_df_his['收盤價'].rolling(backtest_day).min().iloc[-1]
            DAYbacktest_day_max = temp_df_his['收盤價'].rolling(backtest_day).max().iloc[-1]
            backtest_60MAV = temp_df_his['成交筆數'].rolling(backtest_day).mean().iloc[-1]

            temp_df_his = stock_df_raw[
                (stock_df_raw['證券代號'] == i) & 
                (stock_df_raw['Date'] < today_str) & 
                (stock_df_raw['收盤價'] != 0)
            ].copy()


            # 半年前高
            # DAYbacktest_day_long_max = temp_df_his['收盤價'].rolling(backtest_day*2).max().iloc[-1]


            if pd.isna(DAYbacktest_day_min) or pd.isna(DAYbacktest_day_max):  # 檢查是否有 NA
                print(f"DAY240_min = 0 stock_name: {today_df['證券名稱'].iloc[0]} today_str: {today_str} stock_id: {i}")
                continue

            max_min_diff = abs(DAYbacktest_day_max - DAYbacktest_day_min) / DAYbacktest_day_min

            # 符合條件則添加至過濾列表
            if (volumn > backtest_60MAV * 2.5) and (max_min_diff < 0.3):
                stock_id_filter.add(i)  # 加入 stock_id_filter，避免重複處理
                filter_stock_today = stock_df_today[stock_df_today['證券代號'] == i]
                filter_stock_today = filter_stock_today[['Date', '證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '收盤價' ]]
                filter_stock_today['明天開盤價'] = tom_open
                filter_stock_today['明天收盤價'] = tom_close
                filter_stock_today['明今差'] = tom_diff

                filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)


filter_stock_list['Date'] = pd.to_datetime(filter_stock_list['Date'])


insti_inv_raw_sim = insti_inv_raw[['Date', '證券代號','自營商買賣超股數(自行買賣)']]
result_df = pd.merge(filter_stock_list, insti_inv_raw_sim, on=['Date', '證券代號'], how='inner')
result_df2 = result_df[result_df['自營商買賣超股數(自行買賣)'] > 100000]

# 確保 Date 是 datetime 格式
# result_df2['Date'] = pd.to_datetime(result_df2['Date'])


In [ ]:
# stock_df_raw[(stock_df_raw['證券代號'] == '6531') & (stock_df_raw['Date'] >= '2023-11-03	') & (stock_df_raw['收盤價'] != 0)].head(10)

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,60MA,60RS,20RS_sd,20RS_rank,60RS_sd,60RS_rank,240MA,240RS,240RS_sd,240RS_rank
1643586,2023-11-03,6531,愛普*,23403179,19649,9398386209,390.0,410.5,385.5,410.0,...,357.3583,114.7308,1.080863,93.418142,1.147037,93.639381,290.5521,141.1107,1.357885,94.524336
1645395,2023-11-06,6531,愛普*,11786044,11803,4845912689,415.5,420.0,403.5,416.0,...,358.5417,116.0256,1.083325,92.537313,1.149851,93.145384,291.6271,142.6479,1.362287,94.306247
1647204,2023-11-07,6531,愛普*,6866460,6647,2865493801,411.0,421.5,410.0,420.0,...,360.0917,116.6370,1.086075,92.868988,1.153504,93.753455,292.6917,143.4957,1.368700,94.803759
1649013,2023-11-08,6531,愛普*,8197136,10564,3412331644,428.5,428.5,407.0,411.0,...,361.5167,113.6877,1.054076,89.060773,1.120741,91.160221,293.7000,139.9387,1.331499,93.756906
1650822,2023-11-09,6531,愛普*,3976441,3724,1637642221,413.0,415.5,408.5,410.0,...,363.1083,112.9140,1.048143,88.293760,1.113189,91.165102,294.6708,139.1383,1.324578,93.926008
1652633,2023-11-10,6531,愛普*,4682934,4436,1903161001,408.0,411.5,400.0,409.5,...,364.6417,112.3020,1.047835,89.232468,1.111591,91.275538,295.6646,138.5015,1.324592,93.981226
1654444,2023-11-13,6531,愛普*,8221314,7393,3463504792,419.0,425.0,414.5,422.0,...,366.3250,115.1983,1.063542,91.772501,1.130101,93.208172,296.6917,142.2352,1.348646,94.036444
1656255,2023-11-14,6531,愛普*,7031222,6990,3004184070,423.5,435.0,418.0,432.5,...,367.9667,117.5378,1.079082,93.594699,1.148304,94.367753,297.7458,145.2581,1.372067,94.919934
1658066,2023-11-15,6531,愛普*,13203633,11725,5914470730,447.5,459.0,435.5,444.0,...,370.1083,119.9649,1.085969,94.643843,1.158310,95.085588,298.8438,148.5726,1.386850,95.858642
1659876,2023-11-16,6531,愛普*,5902693,7060,2581641648,444.0,445.0,430.5,435.0,...,371.9917,116.9381,1.055524,90.662983,1.127194,92.707182,299.8542,145.0705,1.351744,94.806630


In [16]:
result_df2[result_df2['明今差'] > 0]

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價,明天開盤價,明天收盤價,明今差,自營商買賣超股數(自行買賣)
8,2023-11-03,6531,愛普*,23403179,19649,9398386209,390.00,410.00,415.50,416.00,1.46,255575
16,2023-11-21,3483,力致,28746823,18865,4659464223,158.00,167.00,165.00,176.00,5.39,260000
20,2023-11-24,1513,中興電,55225656,32406,6271656285,110.00,117.00,121.00,119.00,1.71,143919
45,2023-12-15,5483,中美晶,14280086,11501,2808550145,193.00,197.50,196.50,200.50,1.52,246072
47,2023-12-18,2353,宏碁,62047585,27282,2403234171,37.50,38.90,39.10,40.70,4.63,345613
50,2023-12-20,2356,英業達,200187249,95503,9594656154,45.55,49.00,47.50,50.00,2.04,271709
53,2023-12-25,2377,微星,16222880,15622,3115467802,185.00,193.00,194.00,195.50,1.30,198054
56,2023-12-27,2344,華邦電,101704108,35540,3030085767,29.00,29.75,30.20,30.75,3.36,380266
61,2024-01-02,6223,旺矽,12486525,11170,2876883440,223.00,234.00,229.50,239.00,2.14,125936
68,2024-01-18,4746,台耀,16380906,10677,1830846416,108.00,114.50,115.00,116.50,1.75,134000


In [ ]:
# # 定義函數來查找 tom_open 和 tom_close
# def get_tomorrow_prices(row):
#     stock_id = row['證券代號']
#     search_date = row['Date']
#     today_close = row['收盤價']
    
#     # 查找下一個交易日的數據
#     temp_df_tom = stock_df_raw[(stock_df_raw['證券代號'] == stock_id) & (stock_df_raw['Date'] > search_date)].copy()
#     if not temp_df_tom.empty:
#         tom_open = temp_df_tom['開盤價'].iloc[0]
#         tom_close = temp_df_tom['收盤價'].iloc[0]
#         tom_diff = round(100 * (tom_close - today_close) / today_close , 2) 
#         return pd.Series([tom_open, tom_close, tom_diff])
#     else:
#         return pd.Series([None, None])

# # 使用 apply 添加 tom_open 和 tom_close
# result_df2[['tom_open', 'tom_close','tom_diff']] = result_df2.apply(get_tomorrow_prices, axis=1)

# filtered_df = result_df2[result_df2['tom_diff'] > 0].copy()

# filtered_df

In [ ]:
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
day_list = stock_df_raw[(stock_df_raw['Date']>='2024-12-01' ) & (stock_df_raw['Date']<'2024-12-20' )]
day_list = day_list['Date'].unique().tolist()
filter_stock_list = pd.DataFrame(columns=['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價'])

backtest_day = 60

for day in day_list:
        today_str = day.strftime('%Y-%m-%d')
        
        stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

        # 收盤在季線以上 年線以上 成值 10億 
        today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['5MA']) & 
                                    (stock_df_today['5MA']>stock_df_today['10MA']) & 
                                    (stock_df_today['10MA']>stock_df_today['60MA']) & 
                                    (stock_df_today['60MA']>stock_df_today['240MA']) & 
                                    (stock_df_today['成交金額']>1000000000) &
                                    (stock_df_today['成交筆數']>10000) & 
                                    (stock_df_today['60RS_rank']>75) &
                                    (stock_df_today['20RS_rank']>85) ]

        # 計算一年內高點
        stock_id_list = today_list['證券代號'].unique().tolist()

        stock_id_filter = []
        for i in stock_id_list:
                if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天

                        # 計算個股 RS
                        today_df = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] == today_str) ].copy()
                        close = today_df['收盤價'].iloc[0]
                        volumn = today_df['成交筆數'].iloc[0]
                        
                        # 計算個股 RS
                        temp_df_his = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] < today_str) & (stock_df_raw['收盤價'] != 0) ].copy()

                        temp_df_his.loc[:, '60MAV'] = temp_df_his['成交筆數'].rolling(60).mean()

                        # 季內高低點

                        DAYbacktest_day_min = temp_df_his['收盤價'].rolling(backtest_day).min().tail(1)
                        DAYbacktest_day_max = temp_df_his['收盤價'].rolling(backtest_day).max().tail(1)

                        backtest_60MAV = temp_df_his['成交筆數'].rolling(backtest_day).mean().tail(1)


                        # DAY240_money = temp_df_his.tail(1)['backtest_dayDAY_money'].iloc[0]
                        MA20 = temp_df_his.tail(1)['20MA'].iloc[0] 
                        
                        # 計算個股 60日RS rank 從原本小於 80 飆上來 
                        temp_df_his.loc[:, '60RS_rank_mean'] = temp_df_his['60RS_rank'].rolling(30).mean()
                        RS_rank_mean = temp_df_his.tail(1)['60RS_rank_mean'].iloc[0]

                        if (DAYbacktest_day_min == 0) |  (DAYbacktest_day_max == 0) :
                                print('DAY240_min = 0 stock_name:' + temp_df_his['證券名稱'].iloc[0] + "today_str" + today_str + "stock_id" + i)
                        else:
                                max_min_diff = abs(DAYbacktest_day_max - DAYbacktest_day_min) / DAYbacktest_day_min 

                                if( 
                                #(close >  DAYbacktest_day_max) & 
                                (volumn >  backtest_60MAV * 2.5) & 
                                (max_min_diff < 0.3)):
                                        stock_id_filter.append(i) 

        filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
        filter_stock_today = filter_stock_today[['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價']]
        if (len(filter_stock_today)!=0):
                filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)


filter_stock_list


C:\Users\USER\AppData\Local\Temp\ipykernel_932\1413850363.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)


,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價
0,2024-12-02,2383,台光電,13949490,17426,6899354124,474.50,504.00
1,2024-12-02,2419,仲琦,16741317,10276,618945687,35.40,37.05
2,2024-12-02,4303,信立,27298769,14058,1474299157,51.90,55.80
3,2024-12-02,6005,群益證,18017244,7343,422692502,22.85,23.70
4,2024-12-02,6180,橘子,3942634,4577,348621941,87.40,88.40
5,2024-12-03,2610,華航,226943050,60322,5991196836,25.30,26.75
6,2024-12-03,2618,長榮航,94808772,33577,4211802368,43.60,44.60
7,2024-12-03,3380,明泰,9337184,5743,361169335,38.25,38.70
8,2024-12-03,5315,光聯,8860564,4424,211508367,24.00,24.15
9,2024-12-03,5328,華容,17979820,7607,312748311,16.90,17.25


In [156]:
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
day_list = stock_df_raw[(stock_df_raw['Date']>='2023-03-09' ) & (stock_df_raw['Date']<'2023-07-01' )]
day_list = day_list['Date'].unique().tolist()
filter_stock_list = pd.DataFrame(columns=['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價'])
 

for day in day_list:
        today_str = day.strftime('%Y-%m-%d')
        
        stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

        # 收盤在季線以上 年線以上 成值 1億 
        today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['60MA']) & 
                                (stock_df_today['60MA']>stock_df_today['240MA']) & 
                                (stock_df_today['成交金額']>100000000) &
                                (stock_df_today['成交股數']>2000000) & 
                                (stock_df_today['60RS_rank']>75) ]

        # 計算一年內高點
        stock_id_list = today_list['證券代號'].unique().tolist()

        stock_id_filter = []
        for i in stock_id_list:
                if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天

                        # 計算個股 RS
                        today_df = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] == today_str) ].copy()
                        close = today_df['收盤價'].iloc[0]

                        #print(i)
                        # 計算個股 RS
                        temp_df_his = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] < today_str) & (stock_df_raw['收盤價'] != 0) ].copy()
                        # 一年內高低點
                        temp_df_his.loc[:, '120DAY_min'] = temp_df_his['收盤價'].rolling(120).min()
                        temp_df_his.loc[:, '120DAY_max'] = temp_df_his['收盤價'].rolling(120).max()
                        temp_df_his.loc[:, '120DAY_money'] = temp_df_his['成交金額'].rolling(120).mean()


                        DAY120_min = temp_df_his.tail(1)['120DAY_min'].iloc[0]
                        DAY120_max = temp_df_his.tail(1)['120DAY_max'].iloc[0]


                        DAY240_money = temp_df_his.tail(1)['120DAY_money'].iloc[0]
                        MA20 = temp_df_his.tail(1)['20MA'].iloc[0] 
                        
                        # 計算個股 60日RS rank 從原本小於 80 飆上來 
                        temp_df_his.loc[:, '60RS_rank_mean'] = temp_df_his['60RS_rank'].rolling(30).mean()
                        RS_rank_mean = temp_df_his.tail(1)['60RS_rank_mean'].iloc[0]
                        if (DAY120_min == 0) |  (DAY120_max == 0) :
                                print('DAY240_min = 0 stock_name:' + temp_df_his['證券名稱'].iloc[0] + "today_str" + today_str + "stock_id" + i)
                        else:
                                max_min_diff = abs(DAY120_max - DAY120_min) / DAY120_min 

                                # RS_rank_min = 90
                                #距離一年高點25%以內
                                #距離一年低點20%以上
                                if( (close*1.2 >  DAY120_max) & 
                                (close >  DAY120_min*1.2) & 
                                (RS_rank_mean <  50) & 
                                (DAY240_money < 80000000) & 
                                (close < MA20*1.1) & 
                                (DAY240_money < 80000000) &
                                (max_min_diff < 0.3)):
                                        stock_id_filter.append(i) 


        filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
        filter_stock_today = filter_stock_today[['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價']]
        if (len(filter_stock_today)!=0):
                filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)


filter_stock_list


,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價
0,2023-03-10,1339,昭輝,2363188,1769,110362777,44.75,46.45
1,2023-03-15,3038,全台,8123851,3512,198832560,24.35,24.15
2,2023-03-16,4190,佐登-KY,5017107,3930,442868253,85.40,86.10
3,2023-03-21,2387,精元,3767670,2463,162526293,42.60,43.55
4,2023-03-22,2387,精元,2451298,1728,107150720,44.20,43.60
5,2023-03-22,3715,定穎投控,19159920,5584,379071307,19.45,19.55
6,2023-03-28,2331,精英,11548409,4425,273552005,23.55,24.25
7,2023-04-20,3501,維熹,4309832,2896,244953855,55.00,57.60
8,2023-04-21,3501,維熹,4712696,3560,268361560,58.70,55.10
9,2023-04-24,1710,東聯,12152720,4925,255157128,20.40,21.00


In [161]:
today = datetime.now().date()
today.strftime('%Y-%m-%d')

'2024-08-26'

In [2]:
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])

day_list = stock_df_raw[(stock_df_raw['Date']>='2024-08-10' ) & (stock_df_raw['Date']<'2024-10-10' )]
day_list = day_list['Date'].unique().tolist()
filter_stock_list = pd.DataFrame(columns=['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價'])


for day in day_list:
        today_str = day.strftime('%Y-%m-%d')
        stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

        # 收盤在季線以上 年線以上 成值 1億 
        today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['60MA']) & 
                                (stock_df_today['60MA']>stock_df_today['240MA']) & 
                                (stock_df_today['成交金額']>100000000) &
                                (stock_df_today['成交股數']>2000000) & 
                                (stock_df_today['60RS_rank']>75) ]

        # 計算一年內高點
        stock_id_list = today_list['證券代號'].unique().tolist()

        stock_id_filter = []
        for i in stock_id_list:
                if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天

                        # 計算個股 RS
                        close = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] == today_str) ]['收盤價'].iloc[0]

                        #print(i)
                        # 計算個股 RS
                        temp_df_his = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] < today_str) & (stock_df_raw['收盤價'] != 0) ].copy()
                        # 一年內高低點

                        # temp_df_his.loc[:, '120DAY_min'] = temp_df_his['收盤價'].rolling(120).min()
                        # temp_df_his.loc[:, '120DAY_max'] = temp_df_his['收盤價'].rolling(120).max()
                        # temp_df_his.loc[:, '120DAY_money'] = temp_df_his['成交金額'].rolling(120).mean()


                        # DAY120_min = temp_df_his.tail(1)['120DAY_min'].iloc[0]
                        # DAY120_max = temp_df_his.tail(1)['120DAY_max'].iloc[0]
                        temp_df_his_tail = temp_df_his.tail(120)

                        DAY120_min = temp_df_his_tail['收盤價'].min()
                        DAY120_max = temp_df_his_tail['收盤價'].max()


                        DAY240_money = temp_df_his_tail['成交金額'].mean()
                        MA20 = temp_df_his.tail(1)['20MA'].iloc[0] 
                        
                        # 計算個股 60日RS rank 從原本小於 80 飆上來 
                        # temp_df_his.loc[:, '60RS_rank_mean'] = temp_df_his['60RS_rank'].rolling(30).mean()
                        RS_rank_mean = temp_df_his['60RS_rank'].tail(30).mean()
                        if (DAY120_min == 0) |  (DAY120_max == 0) :
                                print('DAY240_min = 0 stock_name:' + temp_df_his['證券名稱'].iloc[0] + "today_str" + today_str + "stock_id" + i)
                        else:
                                max_min_diff = abs(DAY120_max - DAY120_min) / DAY120_min 

                                # RS_rank_min = 90
                                #距離一年高點25%以內
                                #距離一年低點20%以上
                                if( (close*1.2 >  DAY120_max) & 
                                (close >  DAY120_min*1.2) & 
                                (RS_rank_mean <  50) & 
                                (DAY240_money < 80000000) & 
                                (close < MA20*1.1) & 
                                (DAY240_money < 80000000) &
                                (max_min_diff < 0.3)):
                                        stock_id_filter.append(i) 


        filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
        filter_stock_today = filter_stock_today[['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價']]
        if (len(filter_stock_today)!=0):
                filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)


filter_stock_list


C:\Users\USER\AppData\Local\Temp\ipykernel_14192\3959463151.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)


,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價
0,2024-08-12,3003,健和興,3738392,2867,307519785,78.5,82.5
1,2024-08-13,3003,健和興,2646098,3090,220231837,83.4,82.5


In [176]:
# filter_stock_list2023 = filter_stock_list
filter_stock_list2023

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價
0,2023-02-13,6605,帝寶,3053600,2491,245957425,79.30,80.80
1,2023-03-10,1339,昭輝,2363188,1769,110362777,44.75,46.45
2,2023-03-15,3038,全台,8123851,3512,198832560,24.35,24.15
3,2023-03-16,4190,佐登-KY,5017107,3930,442868253,85.40,86.10
4,2023-03-21,2387,精元,3767670,2463,162526293,42.60,43.55
5,2023-03-22,2387,精元,2451298,1728,107150720,44.20,43.60
6,2023-03-22,3715,定穎投控,19159920,5584,379071307,19.45,19.55
7,2023-03-28,2331,精英,11548409,4425,273552005,23.55,24.25
8,2023-04-20,3501,維熹,4309832,2896,244953855,55.00,57.60
9,2023-04-21,3501,維熹,4712696,3560,268361560,58.70,55.10


In [5]:
# Q1 森鉅 撈出來之後如何停損
filter_stock_list

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價
0,2024-07-01,6127,九豪,10116952,4483,305192884,30.15,29.50
1,2024-07-02,6127,九豪,6724807,3949,201901179,29.50,29.80
2,2024-07-02,6261,久元,2342613,1867,191450339,80.10,82.10
3,2024-07-08,1727,中華化,3721227,2517,121341602,33.55,32.75
4,2024-08-01,9940,信義,4225914,2955,151651198,34.20,35.80
5,2024-08-12,3003,健和興,3738392,2867,307519785,78.50,82.50
6,2024-08-13,3003,健和興,2646098,3090,220231837,83.40,82.50


In [22]:
#短側一個月效果不明顯  之後側一年看看
df_unique = filter_stock_list.drop_duplicates(subset='證券代號', keep='first')
print(df_unique)


         Date  證券代號 證券名稱      成交股數  成交筆數       成交金額     開盤價     收盤價
0  2024-01-04  9934   成霖   8658847  3294  140198925   16.50   16.10
1  2024-01-19  6155   鈞寶  15950240  7038  469587912   29.90   28.90
2  2024-02-15  3010   華立   3331012  2748  342529354  100.50  104.50
4  2024-02-16  8110   華東   6665992  2981  106797055   15.60   16.20
7  2024-03-01  3322  建舜電   7902471  4156  202713134   25.00   25.05
9  2024-03-18  2471   資通   6076084  4811  375805025   62.00   61.00
10 2024-03-18  6177   達麗   2974365  1793  112802547   37.60   38.55
11 2024-03-20  2451   創見   2097954  2182  186082787   87.00   88.60
12 2024-04-19  1532   勤美  10242550  5356  427436303   42.80   42.15
15 2024-04-23  3024   憶聲  17102876  5018  324474099   19.15   19.20
17 2024-04-25  8039   台虹   5810045  4143  290490929   49.00   50.50
20 2024-05-23  6166   凌華   3753875  4046  273135064   75.90   71.80
21 2024-06-06  3071   協禧   2929930  2216  119079551   40.00   40.00
22 2024-06-06  8103   瀚荃   2408557  1981  118315

In [30]:
import warnings
# 計算出場  使用60RS看看
# 忽略 FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

RS_down = list(range(75, 76, 1))
earn_type = 0
profolio_df1 = pd.DataFrame(columns=['rs_value', 'mean', 'mean_t1', 'mean_t2','hold_day_mean1','hold_day_mean2','hold_day_mean3','hold_day_mean4'])

for rs_value in RS_down:
    collected_data = pd.DataFrame(columns=['rs_value','stock_id', 'stock_name', 'Date', 
                                           '收盤價', '隔天收盤價', '收盤價_RS小於75','隔天開盤_RS小於75', 
                                           '收盤價_dff','隔日開盤價_dff','收盤價_隔天收盤_dff','收盤價_隔天開盤_dff','收盤價_隔天收盤_adj_dff','收盤價_隔天開盤_adj_dff',
                                           '持有天數1','持有天數2','持有天數3','持有天數4','earn_type'])
    
    for i in range(len(df_unique)):
        temp_df = df_unique.iloc[i].copy()

        filter_day = pd.to_datetime(temp_df['Date'])
        # filter_day30 = filter_day + pd.Timedelta(days=hold_day)
        filter_stock_id = temp_df['證券代號']


        # 篩到之後 隔天一開盤要買 (回測起來 隔天收盤買效果好像更好ㄟ)
        tomorrow_open = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['收盤價'].iloc[0]
        # tomorrow_open = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['開盤價'].iloc[0]

        # 隔日不追高 如果開盤開太高就算了 開盤直接掛昨天收盤價買買看 (好像就是得追高才行呢)
        # if tomorrow_open > temp_df['收盤價']:
        #     continue
        # else :
        #     tomorrow_open = temp_df['收盤價']

        # 找到跌落RS後 不可能那天開盤價可以賣 要重測一次
        RS75_df_t = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d')) & (stock_df_raw['60RS_rank'] < rs_value)].copy()
        
        if RS75_df_t.empty:
            continue
        getout_day = RS75_df_t['Date'].iloc[0]
        RS75_df = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > getout_day.strftime('%Y-%m-%d'))].copy()

        
        # 突破失敗停損 收盤跌破買入價     
        stoploss_df = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d')) & (stock_df_raw['收盤價'] < tomorrow_open)].copy()


        if RS75_df.empty:
            continue

        stock_df_RS750 = RS75_df.iloc[0] # 符合條件當天收盤賣
        stock_df_RS751 = RS75_df.iloc[1] # 符合條件隔天開盤再賣

        diff_day1 = (stock_df_RS750['Date'] - temp_df['Date']).days

        diff1 = round(100 * (stock_df_RS750['收盤價'] / temp_df['收盤價'] - 1), 2) #撈出來當天
        diff_t1 = round(100 * (stock_df_RS750['收盤價'] / tomorrow_open - 1), 2)  #撈出來隔天

        diff_day2 = (stock_df_RS751['Date'] - temp_df['Date']).days

        diff2 = round(100 * (stock_df_RS751['開盤價'] / temp_df['收盤價'] - 1), 2)
        diff_t2 = round(100 * (stock_df_RS751['開盤價'] / tomorrow_open - 1), 2)

        # 
        diff_t3 = diff_t1 
        diff_day3 = diff_day1
        diff_t4 = diff_t2
        diff_day4 = diff_day2

        earn_type = '2'
        # 多判斷是否停損不續抱著
        if not stoploss_df.empty:
                stoploss_day_df = stoploss_df.iloc[0] # 符合條件當天停損
                
                if stoploss_day_df['Date'] < stock_df_RS751['Date']:

                    diff_day_loss = (stoploss_day_df['Date'] - temp_df['Date']).days
                    diff_close_loss = round(100 * (stoploss_day_df['收盤價'] / tomorrow_open - 1), 2) #撈出來當天

                    diff_t3 = diff_t4 = diff_close_loss # 調整停損值
                    diff_day4 = diff_day3 = diff_day_loss # 調整停損天數
                    earn_type = '1'


        result_df = pd.DataFrame({
            'rs_value': [rs_value],
            'stock_id': [filter_stock_id],
            'stock_name': [temp_df['證券名稱']],
            'Date': [temp_df['Date']],
            '收盤價': [temp_df['收盤價']],
            '隔天收盤價': [tomorrow_open],
            '收盤價_RS小於75': [stock_df_RS750['收盤價']],
            '隔天開盤_RS小於75': [stock_df_RS751['收盤價']],
            '收盤價_dff': [diff1],
            '隔日開盤價_dff': [diff2],
            '收盤價_隔天收盤_dff': [diff_t1],
            '收盤價_隔天開盤_dff': [diff_t2],
            '收盤價_隔天收盤_adj_dff': [diff_t3],
            '收盤價_隔天開盤_adj_dff': [diff_t4],
            '持有天數1': [diff_day1],
            '持有天數2': [diff_day2],
            '持有天數3': [diff_day3],
            '持有天數4': [diff_day4],
            'earn_type':[earn_type]
        })

        # 删除全为NA的列，避免可能出现的警告
        result_df.dropna(axis=1, how='all', inplace=True)

        if not result_df.empty:
            collected_data = pd.concat([collected_data, result_df], ignore_index=True)

    # 仅当collected_data不为空时，计算mean并合并到profolio_df中
    if not collected_data.empty:
        mean = collected_data['收盤價_dff'].mean()
        mean_t1 = collected_data['收盤價_隔天收盤_dff'].mean()
        mean_t2 = collected_data['收盤價_隔天開盤_dff'].mean()
        mean_t3 = collected_data['收盤價_隔天收盤_adj_dff'].mean()
        mean_t4 = collected_data['收盤價_隔天開盤_adj_dff'].mean()
        hold_day_mean1 = collected_data['持有天數1'].mean()
        hold_day_mean2 = collected_data['持有天數2'].mean()
        hold_day_mean3 = collected_data['持有天數3'].mean()
        hold_day_mean4 = collected_data['持有天數4'].mean()
        profolio_data = pd.DataFrame({
            'rs_value': [rs_value],
            'mean': [mean],
            'mean_t1': [mean_t1],
            'mean_t2': [mean_t2],
            'mean_t3': [mean_t3],
            'mean_t4': [mean_t4],
            'hold_day_mean1' : [hold_day_mean1],
            'hold_day_mean2' : [hold_day_mean2],
            'hold_day_mean3' : [hold_day_mean3],
            'hold_day_mean4' : [hold_day_mean4]
        })
        profolio_df1 = pd.concat([profolio_df1, profolio_data], ignore_index=True)
        
profolio_df1['pen1'] = profolio_df1['mean']/ profolio_df1['hold_day_mean1']
profolio_df1['pen_t1'] = profolio_df1['mean_t1']/ profolio_df1['hold_day_mean1']

profolio_df1['pen2'] = profolio_df1['mean']/ profolio_df1['hold_day_mean2']
profolio_df1['pen_t2'] = profolio_df1['mean_t2']/ profolio_df1['hold_day_mean2']

profolio_df1['pen_t3'] = profolio_df1['mean_t3']/ profolio_df1['hold_day_mean3']
profolio_df1['pen_t4'] = profolio_df1['mean_t4']/ profolio_df1['hold_day_mean4']

profolio_df1['diff_pen1'] = profolio_df1['mean_t1'] - profolio_df1['mean']
profolio_df1['diff_pen2'] = profolio_df1['mean_t2'] - profolio_df1['mean']

profolio_df1

# 有停損 隔日開盤賣 效果比較好

,rs_value,mean,mean_t1,mean_t2,hold_day_mean1,hold_day_mean2,hold_day_mean3,hold_day_mean4,mean_t3,mean_t4,pen1,pen_t1,pen2,pen_t2,pen_t3,pen_t4,diff_pen1,diff_pen2
0,75,3.204706,2.278235,2.609412,19.058824,20.176471,14.882353,15.117647,3.369412,3.558824,0.168148,0.119537,0.158834,0.129329,0.226403,0.235409,-0.926471,-0.595294


In [13]:
profolio_df1

,rs_value,mean,mean_t1,mean_t2,hold_day_mean1,hold_day_mean2,pen1,pen_t1,pen2,pen_t2,diff_pen1,diff_pen2
0,74,-5.57,-4.5825,-4.3775,17.0,18.0,-0.327647,-0.269559,-0.309444,-0.243194,0.9875,1.1925


0	75	-5.3425	-4.35	-4.3	16.75	17.75	10.25	10.25	-3.4475	-3.4475	-0.318955	-0.259701	-0.300986	-0.242254	-0.336341	-0.336341	

In [32]:
len(collected_data[collected_data['earn_type'] == '1'])
len(collected_data)


17

In [55]:
print('======profolio_df1======diff_pen=================')
print(profolio_df1['diff_pen1'].mean())
print(profolio_df1['diff_pen2'].mean())
profolio_df1

======profolio_df1======diff_pen=================
-0.6657619047619043
2.0593333333333335


,rs_value,mean,mean_t1,mean_t2,hold_day_mean1,hold_day_mean2,pen1,pen_t1,pen2,pen_t2,diff_pen1,diff_pen2
0,60,10.092857,9.429286,9.632857,58.928571,63.357143,0.171273,0.160012,0.159301,0.152041,-0.663571,-0.460000
1,61,9.692857,9.027857,9.892857,58.000000,63.071429,0.167118,0.155653,0.153681,0.156852,-0.665000,0.200000
2,62,9.190714,8.520000,10.737143,50.714286,62.500000,0.181225,0.168000,0.147051,0.171794,-0.670714,1.546429
3,63,7.292857,6.625000,10.817143,41.500000,60.857143,0.175731,0.159639,0.119836,0.177746,-0.667857,3.524286
4,64,7.454286,6.783571,11.076429,41.000000,60.714286,0.181812,0.165453,0.122776,0.182435,-0.670714,3.622143
5,65,7.454286,6.783571,11.172857,41.000000,60.642857,0.181812,0.165453,0.122921,0.184240,-0.670714,3.718571
6,66,7.427143,6.755714,10.468571,40.857143,53.642857,0.181783,0.165350,0.138455,0.195153,-0.671429,3.041429
7,67,8.035714,7.357857,9.972857,40.642857,50.214286,0.197715,0.181037,0.160028,0.198606,-0.677857,1.937143
8,68,8.035714,7.357857,9.842857,40.642857,50.142857,0.197715,0.181037,0.160256,0.196296,-0.677857,1.807143
9,69,8.035714,7.357857,10.112857,40.642857,49.785714,0.197715,0.181037,0.161406,0.203128,-0.677857,2.077143


In [69]:
import warnings
# 計算出場  用持有天數來看看
# 忽略 FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

filter_days = list(range(10, 101, 10)) # 持有天數
# filter_days = list(range(3, 10, 1)) # 持有天數

profolio_df2 = pd.DataFrame(columns=['day', 'mean'])

for hold_day in filter_days:
    collected_data = pd.DataFrame(columns=['stock_id', 'stock_name', 'Date', '收盤價','隔天收盤價', '收盤價_30天', '收盤價_dff','收盤價_隔天收盤_dff'])
    
    for i in range(len(filter_stock_list)):
        temp_df = filter_stock_list.iloc[i].copy()

        filter_day = pd.to_datetime(temp_df['Date'])
        filter_day30 = filter_day + pd.Timedelta(days=hold_day)
        filter_stock_id = temp_df['證券代號']

        filter_day30 = filter_day30.strftime('%Y-%m-%d')

        

        tomorrow_open = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['收盤價'].iloc[0]
        # tomorrow_open = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['開盤價'].iloc[0]
        
        month_df = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] >= filter_day30)].copy()

        if month_df.empty:
            continue

        stock_df_month = month_df.iloc[0]
        diff = round(100 * (stock_df_month['收盤價'] / temp_df['收盤價'] - 1), 2)
        diff_t = round(100 * (stock_df_month['收盤價'] / tomorrow_open - 1), 2)

        result_df = pd.DataFrame({
            'stock_id': [filter_stock_id],
            'stock_name': [temp_df['證券名稱']],
            'Date': [temp_df['Date']],
            '收盤價': [temp_df['收盤價']],
            '隔天收盤價': [tomorrow_open],
            '收盤價_30天': [stock_df_month['收盤價']],
            '收盤價_dff': [diff],
            '收盤價_隔天收盤_dff' : [diff_t],
        })

        # 删除全为NA的列，避免可能出现的警告
        result_df.dropna(axis=1, how='all', inplace=True)

        if not result_df.empty:
            collected_data = pd.concat([collected_data, result_df], ignore_index=True)

    # 仅当collected_data不为空时，计算mean并合并到profolio_df中
    if not collected_data.empty:
        mean = collected_data['收盤價_dff'].mean()
        mean_t = collected_data['收盤價_隔天收盤_dff'].mean()
        profolio_data = pd.DataFrame({
            'day': [hold_day],
            'mean': [mean],
            'mean_t': [mean_t]
        })
        profolio_df2 = pd.concat([profolio_df2, profolio_data], ignore_index=True)
        
profolio_df2['pen'] = profolio_df2['mean']/ profolio_df2['day']
profolio_df2['pen_t'] = profolio_df2['mean_t']/ profolio_df2['day']
profolio_df2['diff_pen'] = profolio_df2['mean_t'] - profolio_df2['mean']
print('======profolio_df2======diff_pen=================')
print(profolio_df2['diff_pen'].mean())
profolio_df2

======profolio_df2======diff_pen=================
2.8942857142857146


,day,mean,mean_t,pen,pen_t,diff_pen
0,3,-2.066667,-0.616667,-0.688889,-0.205556,1.45
1,4,-4.070000,-2.400000,-1.0175,-0.6,1.67
2,5,-10.060000,-6.670000,-2.012,-1.334,3.39
3,6,-8.380000,-4.930000,-1.396667,-0.821667,3.45
4,7,-9.780000,-6.380000,-1.397143,-0.911429,3.40
5,8,-8.660000,-5.220000,-1.0825,-0.6525,3.44
6,9,-8.100000,-4.640000,-0.9,-0.515556,3.46


In [ ]:
import warnings
# 計算出場   融合兩個看看有沒有比較好
# 忽略 FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

# 雙重條件
RS_down = list(range(60, 75, 1))
filter_days = list(range(3, 10, 1)) # 持有天數


profolio_df1 = pd.DataFrame(columns=['rs_value', 'mean', 'mean_t1', 'mean_t2','hold_day_mean1','hold_day_mean2'])

for rs_value in RS_down:
    collected_data = pd.DataFrame(columns=['rs_value','stock_id', 'stock_name', 'Date', '收盤價', '隔天收盤價', '收盤價_RS小於75','隔天開盤_RS小於75', '收盤價_dff','隔日開盤價_dff','收盤價_隔天收盤_dff','收盤價_隔天開盤_dff','持有天數1','持有天數2'])
    
    for i in range(len(df_unique)):
        temp_df = df_unique.iloc[i].copy()

        filter_day = pd.to_datetime(temp_df['Date'])
        # filter_day30 = filter_day + pd.Timedelta(days=hold_day)
        filter_stock_id = temp_df['證券代號']


        # 篩到之後 隔天一開盤要買 (回測起來 隔天收盤買效果好像更好ㄟ)

        tomorrow_open = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['收盤價'].iloc[0]
        # tomorrow_open = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['開盤價'].iloc[0]


        #看RS
        RS75_df = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d')) & (stock_df_raw['60RS_rank'] < rs_value)].copy()

        if RS75_df.empty:
            continue

        # 看持有天數
        month_df = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] >= filter_day30)].copy()

        if month_df.empty:
            continue

        stock_df_month = month_df.iloc[0]



        stock_df_RS750 = RS75_df.iloc[0] # 符合條件當天收盤賣
        stock_df_RS751 = RS75_df.iloc[1] # 符合條件隔天開盤再賣

        diff_day1 = (stock_df_RS750['Date'] - temp_df['Date']).days

        diff1 = round(100 * (stock_df_RS750['收盤價'] / temp_df['收盤價'] - 1), 2) #撈出來當天
        diff_t1 = round(100 * (stock_df_RS750['收盤價'] / tomorrow_open - 1), 2)  #撈出來隔天

        diff_day2 = (stock_df_RS751['Date'] - temp_df['Date']).days

        diff2 = round(100 * (stock_df_RS751['開盤價'] / temp_df['收盤價'] - 1), 2)
        diff_t2 = round(100 * (stock_df_RS751['開盤價'] / tomorrow_open - 1), 2)


        result_df = pd.DataFrame({
            'rs_value': [rs_value],
            'stock_id': [filter_stock_id],
            'stock_name': [temp_df['證券名稱']],
            'Date': [temp_df['Date']],
            '收盤價': [temp_df['收盤價']],
            '隔天收盤價': [tomorrow_open],
            '收盤價_RS小於75': [stock_df_RS750['收盤價']],
            '隔天開盤_RS小於75': [stock_df_RS751['收盤價']],
            '收盤價_dff': [diff1],
            '隔日開盤價_dff': [diff2],
            '收盤價_隔天收盤_dff': [diff_t1],
            '收盤價_隔天開盤_dff': [diff_t2],
            '持有天數1': [diff_day1],
            '持有天數2': [diff_day2]
        })

        # 删除全为NA的列，避免可能出现的警告
        result_df.dropna(axis=1, how='all', inplace=True)

        if not result_df.empty:
            collected_data = pd.concat([collected_data, result_df], ignore_index=True)

    # 仅当collected_data不为空时，计算mean并合并到profolio_df中
    if not collected_data.empty:
        mean = collected_data['收盤價_dff'].mean()
        mean_t1 = collected_data['收盤價_隔天收盤_dff'].mean()
        mean_t2 = collected_data['收盤價_隔天開盤_dff'].mean()
        hold_day_mean1 = collected_data['持有天數1'].mean()
        hold_day_mean2 = collected_data['持有天數2'].mean()
        profolio_data = pd.DataFrame({
            'rs_value': [rs_value],
            'mean': [mean],
            'mean_t1': [mean_t1],
            'mean_t2': [mean_t2],
            'hold_day_mean1' : [hold_day_mean1],
            'hold_day_mean2' : [hold_day_mean2],
        })
        profolio_df3 = pd.concat([profolio_df3, profolio_data], ignore_index=True)
        
profolio_df3['pen1'] = profolio_df3['mean']/ profolio_df3['hold_day_mean1']
profolio_df3['pen_t1'] = profolio_df3['mean_t1']/ profolio_df3['hold_day_mean1']

profolio_df3['pen2'] = profolio_df3['mean']/ profolio_df3['hold_day_mean2']
profolio_df3['pen_t2'] = profolio_df3['mean_t2']/ profolio_df3['hold_day_mean2']

profolio_df3['diff_pen1'] = profolio_df3['mean_t1'] - profolio_df3['mean']
profolio_df3['diff_pen2'] = profolio_df3['mean_t2'] - profolio_df3['mean']

print('======profolio_df3======diff_pen=================')
print(profolio_df3['diff_pen1'].mean())
print(profolio_df3['diff_pen2'].mean())
profolio_df3



In [35]:
profolio_df1

,rs_value,mean,mean_t,hold_day_mean,pen,pen_t,diff_pen
0,60,10.218571,10.404286,61.285714,0.166737,0.169767,0.185714
1,61,10.341429,10.525714,59.857143,0.172768,0.175847,0.184286
2,62,10.341429,10.525714,59.857143,0.172768,0.175847,0.184286
3,63,9.362857,9.544286,52.714286,0.177615,0.181057,0.181429
4,64,10.157143,10.365714,50.000000,0.203143,0.207314,0.208571
5,65,9.742857,9.945714,48.714286,0.200000,0.204164,0.202857
6,66,9.742857,9.945714,48.714286,0.200000,0.204164,0.202857
7,67,9.948571,10.154286,48.571429,0.204824,0.209059,0.205714
8,68,10.204286,10.411429,48.428571,0.210708,0.214985,0.207143
9,69,10.220000,10.424286,46.714286,0.218777,0.223150,0.204286


In [ ]:
###===============================#####
# 算每天新進名單及退出名單

In [6]:
# today = datetime.now().date()
# today_str = today.strftime('%Y-%m-%d')
today_str = '2024-08-15'
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['60MA']) & 
                            (stock_df_today['60MA']>stock_df_today['240MA']) & 
                            (stock_df_today['成交金額']>500000000) &
                            (stock_df_today['成交股數']>2000) &
                            (stock_df_today['60RS_rank']>90)]

# 計算一年內高點
stock_id_list = today_list['證券代號'].unique().tolist()

stock_id_filter = []
for i in stock_id_list:
        if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
                #print(i)
                # 計算個股 RS
                temp_df = stock_df_raw[stock_df_raw['證券代號']==i].copy()
                temp_df.loc[:, '240DAY_min'] = temp_df['收盤價'].rolling(240).min()
                temp_df.loc[:, '240DAY_max'] = temp_df['收盤價'].rolling(240).max()
                DAY240_min = temp_df[temp_df['Date'] == today_str]['240DAY_min'].iloc[0]
                DAY240_max = temp_df[temp_df['Date'] == today_str]['240DAY_max'].iloc[0]
                close = temp_df[temp_df['Date'] == today_str]['收盤價'].iloc[0]
                #距離一年高點25%以內
                #距離一年低點20%以上
                if( (close*1.25 >  DAY240_max) & (close >  DAY240_min*1.2)):
                        stock_id_filter.append(i) 


filter_stock_before = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]



today_str = '2024-08-05'
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
stock_df_today = stock_df_raw[stock_df_raw['Date'] == today_str].copy() 

today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['60MA']) & 
                            (stock_df_today['60MA']>stock_df_today['240MA']) & 
                            (stock_df_today['成交金額']>500000000) &
                            (stock_df_today['成交股數']>2000) &
                            (stock_df_today['20RS_rank']>90) & 
                            (stock_df_today['240RS_rank']<80) ]

# 計算一年內高點
stock_id_list = today_list['證券代號'].unique().tolist()

stock_id_filter = []
for i in stock_id_list:
        if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
                #print(i)
                # 計算個股 RS
                temp_df = stock_df_raw[stock_df_raw['證券代號']==i].copy()
                temp_df.loc[:, '240DAY_min'] = temp_df['收盤價'].rolling(240).min()
                temp_df.loc[:, '240DAY_max'] = temp_df['收盤價'].rolling(240).max()
                DAY240_min = temp_df[temp_df['Date'] == today_str]['240DAY_min'].iloc[0]
                DAY240_max = temp_df[temp_df['Date'] == today_str]['240DAY_max'].iloc[0]
                close = temp_df[temp_df['Date'] == today_str]['收盤價'].iloc[0]
                #距離一年高點25%以內
                #距離一年低點20%以上
                if( (close*1.25 >  DAY240_max) & (close >  DAY240_min*1.2)):
                        stock_id_filter.append(i) 


filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
filter_stock_today

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,60MA,60RS,20RS_sd,20RS_rank,60RS_sd,60RS_rank,240MA,240RS,240RS_sd,240RS_rank
844389,2024-08-05,4904,遠傳,18665389,11170,1617436928,85.4,87.4,84.7,87.4,...,84.6433,103.2568,1.19501,96.758509,1.163385,91.247974,79.4192,110.049,1.054881,77.903836


In [4]:
list_now = list(filter_stock_before['證券代號'])
list_before = list(filter_stock_today['證券代號'])
compare_add = [x for x in list_now if x not in list_before] #新進入名單
compare_remove = [x for x in list_before if x not in list_now] #新進入名單

stock_df_today[stock_df_today['證券代號'].isin(compare_add)]

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,60MA,60RS,20RS_sd,20RS_rank,60RS_sd,60RS_rank,240MA,240RS,240RS_sd,240RS_rank
858295,2024-08-15,1536,和大,7129083,6050,482363043,69.20,69.3,66.30,67.30,...,61.2533,109.8716,1.023210,76.835853,1.124157,90.874730,59.2775,113.5338,0.994284,74.136069
858338,2024-08-15,1708,東鹼,7593898,5117,288844387,38.25,38.9,37.50,37.75,...,34.3508,109.8954,1.081543,92.278618,1.124400,90.928726,32.4377,116.3769,1.019183,76.997840
858502,2024-08-15,2363,矽統,5211819,5947,362582439,70.00,70.5,68.50,69.20,...,60.4983,114.3833,1.085468,92.764579,1.170318,92.926566,49.1663,140.7470,1.232606,91.684665
858725,2024-08-15,2915,潤泰全,5390917,5370,429291891,80.50,81.0,79.00,79.00,...,72.5817,108.8429,1.017685,73.488121,1.113632,89.956803,63.8583,123.7113,1.083414,82.883369
858958,2024-08-15,3535,晶彩科,8056463,6112,523583160,67.00,67.3,63.20,63.80,...,51.0575,124.9572,0.910497,5.183585,1.278506,96.760259,28.0644,227.3345,1.990905,99.082073
859617,2024-08-15,6535,順藥,1824878,3075,462833684,245.00,260.0,242.00,260.00,...,209.8417,123.9029,1.201585,98.272138,1.267719,96.544276,133.8325,194.2727,1.701363,97.732181
859810,2024-08-15,8043,蜜望實,6357707,3307,280450799,42.10,45.1,41.65,45.10,...,27.0892,166.4872,1.451850,99.946004,1.703422,99.568035,26.7038,168.8901,1.479072,95.788337


In [10]:
import collections
df_rs70_category_week = pd.DataFrame()  #多筆每日強勢細產業

sub_category_list.rename(columns={'stock_id': '證券代號'}, inplace=True)

stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
data_date_list = collections.Counter(stock_df_raw['Date'].sort_values(ascending=False))

for day in data_date_list:
        day = day.strftime('%Y-%m-%d')
        # 取每天RS大於90的產業出來
        stock_df_today = stock_df_raw[stock_df_raw['Date']==day].copy()  
        stock_df_today.sort_values(by=['60RS_rank'], inplace=True)

        stock_df_today90 = stock_df_today[stock_df_today['60RS_rank']>90]

        merged_rs_sub = pd.merge(stock_df_today90, sub_category_list, on='證券代號', how='inner')  #RS超過90的
        merged_sub = pd.merge(stock_df_today, sub_category_list, on='證券代號', how='inner')  # 全部股票

        cols = ['Date', '證券代號', 'stock_name','60RS_rank','main_category','sub_category']
        merged_rs_sub = merged_rs_sub[cols]
        merged_sub = merged_sub[cols]

        #找細產業別 跟 母體
        df_rs70_sub_category = collections.Counter(merged_rs_sub['sub_category'])
        df_all_sub_category = collections.Counter(merged_sub['sub_category'])

        df_sub_category = pd.DataFrame()
        for i in df_rs70_sub_category:
                sub_category = {
                "date": [day],
                "sub_category": [i],
                "main_rank": [df_rs70_sub_category[i]],
                "all": [df_all_sub_category[i]],
                "pen": [round((df_rs70_sub_category[i]/df_all_sub_category[i])*100,2)]}

                df_sub_category_temp = pd.DataFrame(sub_category)
                df_sub_category = pd.concat([df_sub_category,df_sub_category_temp])
                df_sub_category.sort_values(by=['pen'], ascending=False, inplace=True)

        df_rs70_category_week = pd.concat([df_rs70_category_week,df_sub_category])

                


In [ ]:
# 個股濾網 (技術面)

#股價大於 60MA

#twse_df_raw
#stock_df_raw
#sub_category_list





